In [1]:
import os
import glob
import pandas as pd
import numpy as np
import re
from math import log2

In [2]:
# Define paths for 0_Experiment_1
base_path = "/Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze"
save_agent_path = "/Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent"
save_face_separated_path = "/Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated"
entropy_log_faces = "/Volumes/TwoTeras/0_Experiment_1/Entropy_Results/entropy_log_faces.csv"
entropy_log_combined = "/Volumes/TwoTeras/0_Experiment_1/Entropy_Results/entropy_log_combined.csv"
failed_files_path = "/Volumes/TwoTeras/0_Experiment_1/Entropy_Results/failed_files_log.csv"
final_matrix_faces_path = "/Volumes/TwoTeras/0_Experiment_1/Entropy_Results/final_transition_matrix_faces.csv"
final_matrix_combined_path = "/Volumes/TwoTeras/0_Experiment_1/Entropy_Results/final_transition_matrix_combined.csv"

# Ensure directories exist
os.makedirs(save_agent_path, exist_ok=True)
os.makedirs(save_face_separated_path, exist_ok=True)

# Define categories
categories_agent = [
    "Background", "Building", "TaskBuilding_Public", "TaskBuilding_Residential",
    "Global_Landmark", "Passive_Agent", "Active_Agent"
]

categories_face_separated = [
    "Background", "Building", "TaskBuilding_Public", "TaskBuilding_Residential",
    "Global_Landmark", "Passive_Agent_Face", "Passive_Agent", "Active_Agent_Face", "Active_Agent"
]

# Mapping dictionary
merge_face_mapping = {
    "Passive_Agent_Face": "Passive_Agent",
    "Active_Agent_Face": "Active_Agent"
}

# Initialize overall raw matrices
overall_raw_matrix_faces = pd.DataFrame(0, index=categories_face_separated, columns=categories_face_separated, dtype=float)
overall_raw_matrix_combined = pd.DataFrame(0, index=categories_agent, columns=categories_agent, dtype=float)

# Initialize entropy results and failure logs
entropy_results_faces = []
entropy_results_combined = []
failed_files = []

# Function to calculate transition entropy
def calculate_transition_entropy(matrix, stationary_distribution):
    total_entropy = 0
    category_entropies = {}
    for i, row in matrix.iterrows():
        row_entropy = sum(-p * np.log2(p) for p in row if p > 0)
        category_entropies[i] = row_entropy
        total_entropy += row_entropy * stationary_distribution.get(i, 0)
    return total_entropy, category_entropies

# Function to process and save data
def process_and_save(file, mapping_type, save_path, all_categories, mapping, entropy_results, overall_matrix):
    try:
        print(f"Processing file: {file} ({mapping_type})")

        # Load the CSV file
        One_participant = pd.read_csv(file)
        if 'Collider_CategoricalN' not in One_participant.columns:
            raise ValueError("Missing required column: 'Collider_CategoricalN'")

        # Filter for desired gaze events
        data_Reduced = One_participant[One_participant['events'] == -2].copy()
        if data_Reduced.empty:
            raise ValueError("Filtered data is empty")

        # Apply mapping if provided
        if mapping:
            data_Reduced['Collider_CategoricalN'] = data_Reduced['Collider_CategoricalN'].replace(mapping)

        # Validate categories in the data
        data_Reduced = data_Reduced[data_Reduced['Collider_CategoricalN'].isin(all_categories)]

        # Build raw transition matrix
        raw_transition_matrix = pd.DataFrame(0, index=all_categories, columns=all_categories, dtype=float)
        gaze_sequence = data_Reduced['Collider_CategoricalN'].reset_index(drop=True)
        for i in range(len(gaze_sequence) - 1):
            current_category = gaze_sequence.iloc[i]
            next_category = gaze_sequence.iloc[i + 1]
            if current_category in all_categories and next_category in all_categories:
                raw_transition_matrix.loc[current_category, next_category] += 1

        # Add raw transitions to the overall matrix
        overall_matrix += raw_transition_matrix

        # Normalize the raw transition matrix
        row_sums = raw_transition_matrix.sum(axis=1)
        normalized_matrix = raw_transition_matrix.div(row_sums.replace(0, 1), axis=0).fillna(0)

        # Calculate stationary distribution
        eigvals, eigvecs = np.linalg.eig(normalized_matrix.T)
        stationary_distribution = np.real(eigvecs[:, np.isclose(eigvals, 1)].flatten())
        stationary_distribution /= stationary_distribution.sum()
        stationary_distribution_dict = {all_categories[i]: stationary_distribution[i] for i in range(len(all_categories))}

        # Calculate transition entropy
        overall_transition_entropy, transition_entropy_per_category = calculate_transition_entropy(normalized_matrix, stationary_distribution_dict)

        # Calculate stationary entropy
        num_categories = len(all_categories)
        stationary_entropy_per_category = {
            category: (-stationary_distribution_dict[category] * np.log2(stationary_distribution_dict[category]))
            if stationary_distribution_dict[category] > 0 else 0
            for category in all_categories
        }

        # Normalize entropies
        normalized_overall_transition_entropy = overall_transition_entropy / np.log2(num_categories) if num_categories > 1 else 0

        # Save transition matrix
        save_matrix_path = os.path.join(save_path, f"{os.path.basename(file).replace('.csv', f'_{mapping_type}_transition_matrix.csv')}")
        raw_transition_matrix.to_csv(save_matrix_path)
        print(f"Saved matrix: {save_matrix_path}")

        # Collect entropy results
        result = {
            'Filename': os.path.basename(file),
            'Overall_Transition_Entropy': normalized_overall_transition_entropy
        }
        result.update({f'Transition_Entropy_{cat}': transition_entropy_per_category.get(cat, 0) / np.log2(num_categories) if num_categories > 1 else 0 for cat in all_categories})
        result.update({f'Stationary_Entropy_{cat}': stationary_entropy_per_category.get(cat, 0) / np.log2(num_categories) if num_categories > 1 else 0 for cat in all_categories})
        entropy_results.append(result)

    except Exception as e:
        error_message = f"Error processing file {file} ({mapping_type}): {e}"
        print(error_message)
        failed_files.append({'filename': os.path.basename(file), 'type': mapping_type, 'error': str(e)})

# Process files and accumulate matrices
files = glob.glob(base_path + "/*.csv")
if not files:
    print(f"No files found in {base_path}")

for file in files:
    process_and_save(file, "Agent", save_agent_path, categories_agent, merge_face_mapping, entropy_results_combined, overall_raw_matrix_combined)
    process_and_save(file, "Face_Separated", save_face_separated_path, categories_face_separated, None, entropy_results_faces, overall_raw_matrix_faces)

# Normalize and save final transition matrices
row_sums_faces = overall_raw_matrix_faces.sum(axis=1)
final_normalized_matrix_faces = overall_raw_matrix_faces.div(row_sums_faces.replace(0, 1), axis=0).fillna(0)
final_normalized_matrix_faces.to_csv(final_matrix_faces_path)
print(f"Final transition matrix (faces separated) saved to: {final_matrix_faces_path}")

row_sums_combined = overall_raw_matrix_combined.sum(axis=1)
final_normalized_matrix_combined = overall_raw_matrix_combined.div(row_sums_combined.replace(0, 1), axis=0).fillna(0)
final_normalized_matrix_combined.to_csv(final_matrix_combined_path)
print(f"Final transition matrix (combined) saved to: {final_matrix_combined_path}")

# Save entropy logs
if entropy_results_faces:
    entropy_df_faces = pd.DataFrame(entropy_results_faces)
    entropy_df_faces.to_csv(entropy_log_faces, index=False)
    print(f"Entropy log (faces separated) saved to: {entropy_log_faces}")

if entropy_results_combined:
    entropy_df_combined = pd.DataFrame(entropy_results_combined)
    entropy_df_combined.to_csv(entropy_log_combined, index=False)
    print(f"Entropy log (combined) saved to: {entropy_log_combined}")

# Log failed files
if failed_files:
    failed_df = pd.DataFrame(failed_files)
    failed_df.to_csv(failed_files_path, index=False)
    print(f"Failures logged to: {failed_files_path}")
else:
    print("All files processed successfully.")


Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/0479_2.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/0479_2_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/0479_2.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/0479_2_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/0479_3.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/0479_3_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/0479_3.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/0479_3_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experimen

Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/2693_1_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/2693_1.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/2693_1_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/2693_2.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/2693_2_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/2693_2.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/2693_2_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/2693_3.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1

Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/3572_4_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/3976_1.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/3976_1_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/3976_1.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/3976_1_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/3976_2.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/3976_2_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/3976_2.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1

Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/4917_3_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/4917_3.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/4917_3_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/4917_4.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/4917_4_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/4917_4.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/4917_4_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/4917_5.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1

Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/6642_2_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/6642_3.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/6642_3_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/6642_3.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/6642_3_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/6642_4.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/6642_4_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/6642_4.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1

Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/7412_5_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/7412_5.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/7412_5_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/7842_1.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/7842_1_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/7842_1.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/7842_1_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/7842_2.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1

Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/8673_2_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/8673_3.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/8673_3_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/8673_3.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/8673_3_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/8673_4.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/8673_4_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/8673_4.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1

Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/9601_2_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/9601_2.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/9601_2_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/9601_3.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/9601_3_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/9601_3.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/9601_3_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/9601_4.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1

Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/9502_4_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/9586_1.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/9586_1_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/9586_1.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Face_Separated/9586_1_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/9586_2.csv (Agent)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Matrices/Agent/9586_2_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/05_Debbies_gaze/9586_2.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/0_Experiment_1